In [82]:
import pandas as pd 
data = pd.read_csv('heart_disease_3.csv')
data = data.dropna()
data['BMI'] = data['BMI'].astype('float')

for col_name in data.columns:
    if(data[col_name].dtype == 'object'):
        data[col_name]= data[col_name].astype('category')
        data[col_name] = data[col_name].cat.codes

/tmp/ipykernel_5271/3665971575.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('heart_disease_3.csv')


In [83]:
from sklearn.model_selection import train_test_split
y = data.pop('HeartDisease')
X = data

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [88]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]


# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


## performance on test data
test_data_prediction = grid_result.best_estimator_.predict(X_test)
test_accuracy = accuracy_score(y_test, test_data_prediction)
print("Test Data accuracy: ", test_accuracy)

Best: 0.915431 using {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}
0.915175 (0.000640) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.915066 (0.000756) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.915171 (0.000634) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.915176 (0.000639) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.915024 (0.000703) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.915176 (0.000636) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.915174 (0.000634) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.914959 (0.000746) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.915179 (0.000640) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.915170 (0.000627) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.915118 (0.000757) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.915243 (0.000628) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.915188 (0.000595) wi